In [3]:
%conda install -c anaconda libev
%conda install -c msys2 m2-make
%conda install -c conda-forge pkg-config
%pip install cassandra-driver

Channels:
 - anaconda
 - defaults
Platform: win-64
Solving environment: failed

Note: you may need to restart the kernel to use updated packages.



PackagesNotFoundError: The following packages are not available from current channels:

  - libev

Current channels:

  - https://conda.anaconda.org/anaconda
  - defaults

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




Channels:
 - msys2
 - defaults
Platform: win-64
Solving environment: done

## Package Plan ##

  environment location: c:\Users\Jvnc\Documents\BD2\BD2-Labs\Lab15\.conda

  added / updated specs:
    - m2-make


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    m2-bash-4.3.042            |                5         3.5 MB  msys2
    m2-gcc-libs-5.3.0          |                4        1021 KB  msys2
    m2-libiconv-1.14           |                3         716 KB  msys2
    m2-libintl-0.19.7          |                4          22 KB  msys2
    m2-make-4.1                |                5         454 KB  msys2
    m2-msys2-runtime-2.5.0.17080.65c939c|                3         3.0 MB  msys2
    msys2-conda-epoch-20160418 |                1           2 KB  msys2
    ------------------------------------------------------------
                                           Total:         8.7 MB

T



==> WARNING: A newer version of conda exists. <==
    current version: 25.3.1
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c defaults conda




Channels:
 - conda-forge
 - defaults
Platform: win-64
Solving environment: done

## Package Plan ##

  environment location: c:\Users\Jvnc\Documents\BD2\BD2-Labs\Lab15\.conda

  added / updated specs:
    - pkg-config


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    gettext-0.21.1             |       h5728263_0         5.3 MB  conda-forge
    libglib-2.68.4             |       h3be07f2_1         3.3 MB  conda-forge
    libiconv-1.17              |       h8ffe710_0         698 KB  conda-forge
    pcre-8.45                  |       h0e60522_0         518 KB  conda-forge
    pkg-config-0.29.2          |    h2bf4dc2_1008          33 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         9.9 MB

The following NEW packages will be INSTALLED:

  gettext            conda-forge/win-64::gettext-0.21.1-h5728263_0 




==> WARNING: A newer version of conda exists. <==
    current version: 25.3.1
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c defaults conda





   ---------------------------------------- 0/3 [click]
   -------------------------- ------------- 2/3 [cassandra-driver]
   -------------------------- ------------- 2/3 [cassandra-driver]
   -------------------------- ------------- 2/3 [cassandra-driver]
   -------------------------- ------------- 2/3 [cassandra-driver]
   ---------------------------------------- 3/3 [cassandra-driver]

Note: you may need to restart the kernel to use updated packages.
